# Nama : Adz Dzikry Pradana Putra210411100093
#Kelas : Penambangan Web A

# Tugas 1 : Crawling Berita

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# Fungsi untuk membersihkan konten dari elemen-elemen yang tidak diinginkan
def clean_content(content_element):
    if content_element:
        # Hapus elemen yang berisi daftar isi
        for daftar_isi in ["collapsible"]:
            unwanted = content_element.find("div", id=daftar_isi)
            if unwanted:
                unwanted.decompose()

        # Hapus elemen yang berisi tag
        for tag_class in ["aevp", "detail__body-tag mgt-16"]:
            unwanted = content_element.find_all("div", class_=tag_class)
            for el in unwanted:
                el.decompose()

        # Hapus elemen yang berisi link sisipan
        link_sisip = content_element.find_all("table", class_="linksisip")
        for table in link_sisip:
            table.decompose()

        # Hapus elemen paragraf dan span dengan class 'para_caption'
        unwanted_paragraphs = content_element.find_all(["p", "span"], class_="para_caption")
        for para in unwanted_paragraphs:
            para.decompose()

        # Kembalikan teks yang tersisa
        return content_element.get_text(separator=' ', strip=True).strip()

    return "Content Not Found"

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
    except requests.exceptions.RequestException as e:
        print(f"Request failed for URL {url}: {e}")
        return

    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link, timeout=10)
            article_response.raise_for_status()

            article_soup = BeautifulSoup(article_response.content, "html.parser")
            title_element = article_soup.find("h1", class_="detail__title")
            title = title_element.text.strip() if title_element else "Title Not Found"
            date_element = article_soup.find("div", class_="detail__date")
            date = date_element.text.strip() if date_element else "Date Not Found"
            content_element = article_soup.find("div", class_="detail__body-text")
            content = clean_content(content_element)

            judul.append(title)
            tanggal.append(date)
            isi.append(content)

            print(f"Scraped: {title}")
            article_response.close()

        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request failed for article {link}: {e}")
            continue

        time.sleep(random.uniform(1, 3))  # Menambahkan jeda waktu 1-3 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://inet.detik.com/games/indeks"]
categories = ["Games"]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    for page in range(1, 4):  # Looping untuk beralih halaman
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            break

        url = f"{base_url}/{page}"
        print(f"Fetching URL: {url}")
        get_data(url, category)
        time.sleep(random.uniform(2, 5))  # Menambahkan jeda waktu 2-5 detik antara permintaan halaman

# Membuat dataframe dari list data
df = pd.DataFrame({
    "Judul": judul,
    "Tanggal": tanggal,
    "Isi": isi
})

# Menyimpan dataframe ke file CSV
try:
    df.to_csv("Crawl-berita.csv", index=False)
    print("Data successfully saved to 'Crawl-berita.csv'")
except Exception as e:
    print(f"Failed to save data: {e}")


In [13]:
df=pd.read_csv("Crawl-berita.csv")
df

,Judul,Tanggal,Isi
0,"3 Game Gratis PS Plus di PS4 dan PS5, Ada Litt...","Selasa, 03 Sep 2024 10:05 WIB",Jakarta - Sony PlayStation akhirnya mengungkap...
1,"Cheat GTA San Andreas Bahasa Indonesia di PS4,...","Selasa, 03 Sep 2024 08:39 WIB",Jakarta - Ini dia Cheat GTA San Andreas Bahasa...
2,"Daftar Game PS4, PS5, Xbox, Nintendo Switch, d...","Selasa, 03 Sep 2024 07:37 WIB","Jakarta - Sudah masuk September nih, kira-kira..."
3,Hasil dan Klasemen FFWS SEA 2024 Fall Week 3: ...,"Senin, 02 Sep 2024 16:10 WIB",Jakarta - Pekan ketiga babak knockout stage FF...
4,"10 Game Offline Seru 2024 untuk HP, Gratis dan...","Senin, 02 Sep 2024 13:19 WIB",Jakarta - Game offline selalu jadi pilihan fav...
5,Bangga! Indonesia Juara FIFAe World Cup 2024 F...,"Senin, 02 Sep 2024 09:45 WIB",Jakarta - Indonesia menjadi juara dunia FIFAe ...
6,"Game Forsaken World 2 Segera Rilis, Ini Cara D...","Minggu, 01 Sep 2024 21:30 WIB",Jakarta - Game Forsaken World 2 mulai membuka ...
7,"World Cyber Games 2024 Festival Digelar, Ada A...","Minggu, 01 Sep 2024 18:00 WIB",Jakarta - Turnamen esports internasional WCG (...
8,"Daftar Cheat GTA 5 di PS3, PS4, PS5, Xbox, dan PC","Minggu, 01 Sep 2024 16:02 WIB",Jakarta - Siapa yang lagi cari cheat GTA 5 nih...
9,Jadwal MPL ID S14 Week 4 Hari Ini: Geek Fam Vs...,"Minggu, 01 Sep 2024 12:02 WIB",Jakarta - Babak regular season MPL ID S14 peka...
